<a href="https://colab.research.google.com/github/rethamlai/texas-holdem-poker/blob/main/poker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import random
from itertools import groupby
from operator import itemgetter

In [3]:
# Card parameters
nCards = 52
nSuits = 4
nNumbers = 13

In [4]:
# Card deck
cardDeck = pd.DataFrame(list(range(1,nCards+1)),columns=['Number'])
cardDeck['Value'] = pd.DataFrame(list(range(1,nNumbers+1))*nSuits)
cardDeck['SuitValue'] = (cardDeck['Number'] - cardDeck['Value'])/nNumbers+1

In [5]:
def draw_card(listVal, nPlayers, temp):
  cards = random.sample(listVal, nPlayers)
  temp.drop([x-1 for x in cards], 0, inplace=True)
  return cards

In [11]:
def play(nPlayers, cardDeck):

    nPlayers = nPlayers
    
    #Card deck copy
    temp = cardDeck.copy()
    
    # Player hands
    playerHands = draw_card(list(temp['Number']), 2 * nPlayers, temp)
    
    # Flop
    burnFlop = draw_card(list(temp['Number']), 1, temp)
    drawFlop = draw_card(list(temp['Number']), 3, temp)
    commCard = drawFlop.copy()
    
    # Turn
    burnTurn = draw_card(list(temp['Number']), 1, temp)
    drawTurn = draw_card(list(temp['Number']), 1, temp)
    commCard.append(drawTurn[0])
    
    # River
    burnRiver = draw_card(list(temp['Number']), 1, temp)
    drawRiver = draw_card(list(temp['Number']), 1, temp)
    commCard.append(drawRiver[0])
    
    return playerHands, commCard, burnFlop, drawFlop, burnTurn, drawTurn, burnRiver, drawRiver

In [7]:
def check_straight_flush(hand):
    try:
        if (check_flush(hand) == True & check_straight(hand) == True):
            return True
        else:
            return False
    except Exception:
        return False

def check_four_of_a_kind(hand):
    valCount = hand.groupby(['Value']).count()
    try:
        valCount.index[valCount['Number'] == 4][0]
        return True
    except Exception:
        return False

def check_full_house(hand):
    valCount = hand.groupby(['Value']).count()
    try:
        valCount.index[valCount['Number'] == 2][0]
        valCount.index[valCount['Number'] == 3][0] 
        return True
    except Exception:
        return False

def check_flush(hand):
    suitCount = hand.groupby(['SuitValue']).count()
    try:
        suitCount.index[suitCount['Number'] >= 5][0]
        return True
    except Exception:
        return False
            
def check_regular_straight(hand):
    uniqueList = sorted(hand['Value'].unique().tolist())
    passed = []
    for k, g in groupby(enumerate(uniqueList), lambda ix : ix[0] - ix[1]):
        consec = list(map(itemgetter(1), g))
        passed.append(len(consec))
        
    if (5 in passed):
        return True
    elif (6 in passed):
        return True
    elif (7 in passed):
        return True
    else:
        return False

def check_low_straight(hand):
    uniqueList = hand['Value'].unique().tolist()
    passedAce = []
    for n, i in enumerate(uniqueList):
        if i == 1:
            replaceAceList = uniqueList.copy()
            replaceAceList[n] = 14   
            replaceAceList = sorted(replaceAceList) 
    try:
        for k, g in groupby(enumerate(replaceAceList), lambda ix : ix[0] - ix[1]):
            consecAce = list(map(itemgetter(1), g))
            passedAce.append(len(consecAce))
                
        if (5 in passedAce):
            return True
        elif (6 in passedAce):
            return True
        elif (7 in passedAce):
            return True
        else:
            return False
    except Exception:
        return False
    
def check_straight(hand):
    try:
        if (check_regular_straight(hand) == True):
            return True
        elif (check_low_straight(hand) == True):
            return True
        else: 
            return False
    except Exception:
        return False

def check_three_of_a_kind(hand):
    valCount = hand.groupby(['Value']).count()
    try:
        valCount.index[valCount['Number'] == 3][0]
        return True
    except Exception:
        return False

def check_two_pairs(hand):
    valCount = hand.groupby(['Value']).count()
    try:
        valCount.index[valCount['Number'] == 2][0]
        valCount.index[valCount['Number'] == 2][1]
        return True
    except Exception:
        return False

def check_one_pairs(hand):
    valCount = hand.groupby(['Value']).count()
    try:
        valCount.index[valCount['Number'] == 2][0]
        return True
    except Exception:
        return False

def check_hand(hand):
    if check_straight_flush(hand):
        return 9
    if check_four_of_a_kind(hand):
        return 8
    if check_full_house(hand):
        return 7
    if check_flush(hand):
        return 6
    if check_straight(hand):
        return 5
    if check_three_of_a_kind(hand):
        return 4
    if check_two_pairs(hand):
        return 3
    if check_one_pairs(hand):
        return 2
    return 1

In [8]:
def check_winner(nPlayers, playerHands, commCard):
    playerScores = []
    handCombined = pd.DataFrame()
    for n in range(0, nPlayers):
        hand = commCard.copy()
        for i in range(0,2):
            hand.append(playerHands[n::nPlayers][i])
        hand = cardDeck.loc[[x-1 for x in hand]]
        hand['Player'] = n
        handCombined = pd.concat([handCombined, np.transpose(hand.reset_index())], axis=1)
        playerScores.append(check_hand(hand))
        
    playerScores.count(max(playerScores))
    
    
    for p in range(0, len(playerScores)):
        print('Player ' + str(p) + ' | Score of: ' + str(playerScores[p]))
        print([i for i, x in enumerate(playerScores) if x == max(playerScores)])

    return np.transpose(handCombined), [i for i, x in enumerate(playerScores) if x == max(playerScores)]

In [13]:
nPlayers = 3
playerHands, commCard, burnFlop, drawFlop, burnTurn, drawTurn, burnRiver, drawRiver = play(nPlayers, cardDeck)
handCombined, playerScores = check_winner(nPlayers, playerHands, commCard)

Player 0 | Score of: 2
[0, 1, 2]
Player 1 | Score of: 2
[0, 1, 2]
Player 2 | Score of: 2
[0, 1, 2]
